<a href="https://colab.research.google.com/github/harshnadar/ISB-intern-work/blob/main/earnings_call.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import glob
import os
from dateparser.search import search_dates

In [ ]:
#set the directory
%cd "C:\Users\HP\Documents\GitHub\intern ISB"

C:\Users\HP\Documents\GitHub\intern ISB


In [ ]:
from bs4 import BeautifulSoup
import re

In [ ]:
#index of capital letters and used further to extract the name and designation
def index(first_text):
    c=[]
    for match in re.finditer(r'[A-Z]{2,5}\W{0,2}:',first_text):
        a=match.span()[0]
        b=match.span()[1]
        c.append([a,b])
    for i in range(len(c)):
        d=c[i][0]
        while(1):
            d=d-1
            if(first_text[d]=='\n'):
                break
        c[i][0]=d
    name=[]
    for i in range(len(c)):
        f=c[i][0] 
        while(1):
            f=f+1
            if(first_text[f]==':'):
                break
            if(first_text[f]==','):
                break
        name.append([c[i][0],f])
    return c, name

In [ ]:
#a dictionary that has name-designation as the key-value pair and des is the list that has the designation of the names. 
#returns a list
def name_designation(first_text, c, name, final_text, name_): 
    b_dict={}
    for i in range(len(c)):
        if(name[i][0]==c[i][0] and name[i][1]!=c[i][1]-1):
            b_dict[first_text[name[i][0]:name[i][1]]]= [first_text[(name[i][1]+2):(c[i][1]-1)]]
    for name in name_:
        if '\n'+name not in b_dict.keys():
            b_dict['\n'+name]=None
    des=[]
    for key in name_:
            des.append(b_dict['\n'+key])
    
    return des

In [ ]:
#dictionary that has name-text as the key-value pair
#returns a final speech and name of the participants
def name_text_dict(first_text,c , name):
    a_dict = {}
    for i in range(len(c)):
        if(i==(len(c)-1)):
            if first_text[name[i][0]:name[i][1]] in a_dict:
                a_dict[first_text[name[i][0]:name[i][1]]].append(first_text[c[i][1]:-1])
            else:
                a_dict[first_text[name[i][0]:name[i][1]]] = [first_text[c[i][1]:-1]]

        else:
            if first_text[name[i][0]:name[i][1]] in a_dict:
                a_dict[first_text[name[i][0]:name[i][1]]].append(first_text[c[i][1]:c[i+1][0]])
            else:
                a_dict[first_text[name[i][0]:name[i][1]]] = [first_text[c[i][1]:c[i+1][0]]]
    final_text=[]
    for key in a_dict.keys():
        final_text.append(" ".join(a_dict[key]))
    name_=[]
    for key in a_dict.keys():
        name_.append(key.split('\n')[1])
    return final_text, name_

In [ ]:
#returns the list that has title of the call
def head(first_text,final_text):
    heading = re.findall('\n(.*).?\n',first_text)
    title=heading[0]
    if(title==''):
        heading=re.findall('\n\n(.*).?\n',first_text)
        title= heading[0]
        dinank=re.findall('\n(.*).?\n',first_text)
        string_1= re.sub(r"([0-9]+(\.[0-9]+)?)",r" \1 ", dinank[1]).strip()
    else:
        dinank=re.findall('\n(.*).?\n\n',first_text)
        string_1=re.sub(r"([0-9]+(\.[0-9]+)?)",r" \1 ", dinank[0]).strip()
    title_=[]
    date=[]
    for i in range(0,len(final_text)):
        title_.append(title)
        try:
            date.append(search_dates(string_1)[1][0])
        except:
            heading=re.findall('(.*).?\n',first_text)
            string_1=re.sub(r"([0-9]+(\.[0-9]+)?)",r" \1 ", heading[2]).strip()
            try:
                date.append(search_dates(string_1)[1][0])
            except:
                date.append(" ")
    return title_, date

In [ ]:
#main function
t=[]
z=[]
n=[]
d=[]
s=[]
v=[]
for i in glob.glob("..\\Prashant Shukla - harsh/*"):
    name_of_folder=i.split('\\')[-1]
    #print(name)
    j=i+"/*.html"
    for k in glob.glob(j): 
        print(k)
        bs=BeautifulSoup(open(k, encoding="utf8"),features="lxml")
        my=bs.findAll("div",{"class": "article"})
        q=0
        while(q<len(my)):
            first_text=my[q].text
            c, name= index(first_text)
            final_text, name_ = name_text_dict(first_text,c , name)
            des= name_designation(first_text, c, name, final_text, name_)
            title_, date_ = head(first_text,final_text)
            v.extend(name_of_folder)
            t.extend(title_)
            z.extend(date_)
            n.extend(name_)
            d.extend(des)
            s.extend(final_text)
            q+=2

..\Prashant Shukla - harsh\10016\1.html
..\Prashant Shukla - harsh\10016\2.html


KeyboardInterrupt: 

In [ ]:
df=pd.DataFrame({'Title':t, 'Date':z, 'Name': n, 'Designation': d, 'Speech':s})

In [ ]:
len(t)

29

In [ ]:
df.tail()

,Title,Date,Name,Designation,Speech
24,Q1 2007 Smurfit-Stone Container Corporation Ea...,27 April 2007,CLAUDIA SHANK,"[ANALYST, JPMORGAN]",Thanks very much. Good morning. Just buil...
25,Q1 2007 Smurfit-Stone Container Corporation Ea...,27 April 2007,PETER RUSCHMEIER,"[ANALYST, LEHMAN BROTHERS]",Thank you. And good morning. I had some relat...
26,Q1 2007 Smurfit-Stone Container Corporation Ea...,27 April 2007,RICK SKIDMORE,"[ANALYST, GOLDMAN SACHS]","Good morning, thank you. Just to follow-up on..."
27,Smurfit-Stone Container Corporation Analyst Co...,,PAT MOORE,"[PRESIDENT AND CEO, SMURFIT-STONE CONTAINER CO...","\nGood morning and thank you, Matt (ph), for t..."
28,Smurfit-Stone Container Corporation Analyst Co...,,UNIDENTIFIED,None,Thanks Pat. Why don't we take a couple of q...


In [ ]:
#set path and change the name of the excel file(use .xlsx format only)
df.to_excel (r'C:\Users\HP\Documents\GitHub\intern ISB\harsh_final.xlsx', index = False, header=True)